# Rappels jour 2 

* gotchas sur les fonctions 
    * pas d'arguments mutables en entré
    * accès aux varaible : L(ocal) E(nglobing) G(lobal) B(uiltin)
    * les variables englobing sont passées par closure
    * en python les closures : 
        * sont late binding 
        * sont en lecture seule
* curryfication 
* décorateurs